In [1]:
!pip install pandas numpy

In [2]:
import pandas as pd

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

/home/de-ninja/airflow-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████████████████████████████████████████████████████████████| 25.7M/25.7M [00:03<00:00, 7.65MB/s]

Extracting files...


Path to dataset files: /home/de-ninja/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [7]:
df = pd.read_csv(path+'/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
df.loc[1][0]

/tmp/ipykernel_3990/2044023174.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[1][0]


'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

#### 1. Removing HTML Tags

In [10]:
import re
def remove_html_tags(text):
    pattern = re.compile("<.*?>")
    return pattern.sub(r'', text)

In [16]:
df['review'] = df['review'].apply(remove_html_tags)

In [18]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#### 2. Removing URLs

In [32]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+')
    return pattern.sub(r'', text)

In [35]:
df['review'] = df['review'].apply(remove_urls)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#### 3. Puntuation Handling

In [36]:
import string, time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [43]:
text = "This is Rohit Kumar, an AI enthusiast. "

In [44]:
def remove_punctuations(text):
    for i in text:
        if i in string.punctuation:
            text = text.replace(i, '')
    return text

In [46]:
start = time.time()
# remove_punctuations(text)
df['review'] = df['review'].apply(remove_punctuations)
end = time.time()
end - start

8.524785280227661

In [47]:
def remove_punctuations_v2(text):
    return text.translate(str.maketrans('','',string.punctuation))

In [48]:
start = time.time()
# remove_punctuations(text)
df['review'] = df['review'].apply(remove_punctuations_v2)
end = time.time()
end - start

1.7560343742370605

#### 4. Chat Abbreviations Handling

In [49]:
chat_abbreviations = {
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "BRB": "Be Right Back",
    "BTW": "By The Way",
    "DM": "Direct Message",
    "FYI": "For Your Information",
    "GG": "Good Game",
    "GLHF": "Good Luck, Have Fun",
    "IDK": "I Don’t Know",
    "IMO": "In My Opinion",
    "IMHO": "In My Humble Opinion",
    "IRL": "In Real Life",
    "LMAO": "Laughing My Ass Off",
    "LOL": "Laugh Out Loud",
    "NP": "No Problem",
    "NSFW": "Not Safe For Work",
    "OMG": "Oh My God",
    "OMW": "On My Way",
    "ROFL": "Rolling On Floor Laughing",
    "SMH": "Shaking My Head",
    "TBH": "To Be Honest",
    "TTYL": "Talk To You Later",
    "TY": "Thank You",
    "YOLO": "You Only Live Once",
    "JK": "Just Kidding",
    "TMI": "Too Much Information",
    "WFH": "Work From Home",
    "LMK": "Let Me Know",
    "IKR": "I Know, Right?",
    "RN": "Right Now",
    "BFF": "Best Friends Forever",
    "FOMO": "Fear Of Missing Out",
    "GTG": "Got To Go",
    "IDC": "I Don’t Care",
    "ETA": "Estimated Time of Arrival",
    "HMU": "Hit Me Up",
    "WC": "Welcome (or Water Closet)",
    "YOLO": "You Only Live Once"
}

In [58]:
def replacing_chat_abbreviations(text):
    new_text = []
    for word in text.split(' '):
        if word.upper() in chat_abbreviations.keys():
            new_text.append(chat_abbreviations[word.upper()])
        else:
            new_text.append(word)
    return ' '.join(new_text)

In [60]:
t = 'Do this work ASAP'
replacing_chat_abbreviations(t).lower()

'do this work as soon as possible'

#### 5. Incorrect Spelling Handling

In [62]:
from textblob import TextBlob

In [65]:
from textblob import TextBlob
incorrect_text = 'Tomorow I wil definitly recieve a mesage about thier suprise plan.'

textblobObj = TextBlob(incorrect_text)

textblobObj.correct().string

'Tomorrow I will definitely receive a message about their surprise plan.'

#### 6. Stopwords Handling

In [66]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/de-
[nltk_data]     ninja/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [72]:
stopwords.words('english')[:5]

['i', 'me', 'my', 'myself', 'we']

In [73]:
def remove_stopwords(text):
    new_text = []
    for word in text.split(' '):
        if word not in stopwords.words('english'):
            new_text.append(word)
    return new_text

In [74]:
string = "This is a good man"

remove_stopwords(string)

['This', 'good', 'man']

#### 7. Handling Emojis

In [76]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile(
        r"[\U0001F600-\U0001F64F]"  # Emoticons
        r"|[\U0001F300-\U0001F5FF]"  # Miscellaneous Symbols and Pictographs
        r"|[\U0001F680-\U0001F6FF]"  # Transport and Map Symbols
        r"|[\u2600-\u26FF]"          # Miscellaneous Symbols
        r"|[\u2700-\u27BF]"          # Dingbats
        r"|[\U0001F900-\U0001F9FF]"  # Supplemental Symbols and Pictographs
        r"|[\U0001FA70-\U0001FAFF]"  # Symbols and Pictographs Extended
        r"|[\U0001F1E6-\U0001F1FF]"  # Regional Indicator Symbols
        r"|[\U0001F200-\U0001F2FF]"  # Enclosed Ideographic Supplement
        r"|[\U0001F7E0-\U0001F7FF]"  # Geometric Shapes Extended
        r"|[\U0001F100-\U0001F1FF]"  # Enclosed Alphanumeric Supplement
        r"|[\U0001F3FB-\U0001F3FF]"  # Skin tone modifiers
    )

    return emoji_pattern.sub('', text)

# Example usage
text = "Hello 🌍! Check out this 🚀 and 😂!"
remove_emojis(text)

'Hello ! Check out this  and !'

In [80]:
import emoji

emoji.demojize('This movie is 🔥')

'This movie is :fire:'

#### 8. Tokenisation

##### 8.a Split

In [81]:
# Word Tokenisation
sentence = "My name is Rohit"
sentence.split(' ')

['My', 'name', 'is', 'Rohit']

In [82]:
# Sentence Tokernisation
para = 'Hi, my name is Rohit. I am working as a data engineer'
para.split('.')

['Hi, my name is Rohit', ' I am working as a data engineer']

In [84]:
# Problem with split function for tokenisation
para = 'Hi! my name is Rohit!'
para.split(' ')

['Hi!', 'my', 'name', 'is', 'Rohit!']

##### 8.b Regex

In [88]:
# Tokenisation with Regex
import re
sentence = 'Hi I am Rohit!'
tokens = re.findall(r"[\w']+", sentence)
tokens

['Hi', 'I', 'am', 'Rohit']

##### 8.c NLTK

In [93]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/de-ninja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/de-
[nltk_data]     ninja/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [94]:
sent1 = "Hi My name is Rohit!"
word_tokenize(sent1)

['Hi', 'My', 'name', 'is', 'Rohit', '!']

In [97]:
para = """As the sun dipped below the horizon, painting the sky in hues of orange and pink, the children laughed and played in the park. Their joyful voices echoed through the air, blending with the soft rustling of leaves. Meanwhile, parents sat on benches, sharing stories and watching the evening unfold peacefully."""
sent_tokenize(para)

['As the sun dipped below the horizon, painting the sky in hues of orange and pink, the children laughed and played in the park.',
 'Their joyful voices echoed through the air, blending with the soft rustling of leaves.',
 'Meanwhile, parents sat on benches, sharing stories and watching the evening unfold peacefully.']

##### 8.d Spacy (Good)

In [104]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [106]:
import spacy
npl = spacy.load('en_core_web_sm')

In [116]:
sent = "Hi, I am Rohit!"
npl(sent)

Hi, I am Rohit!

In [109]:
# Spacy is better as it do the following things:
# 1. Tokenization: The text is split into individual tokens (words, punctuation marks, etc.).
# 2. Part-of-speech tagging (POS): Each token is assigned a part-of-speech label (e.g., noun, verb).
# 3. Named Entity Recognition (NER): The model identifies named entities like person names, organizations, and locations.
# 4. Dependency parsing: The relationships between tokens (which words are subjects, objects, etc.) are identified

#### 9. Stemming

In [118]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
def stem_words(text):
    return ' '.join([ps.stem(word) for word in text.split()])

In [124]:
sample = "walks walk walked walking"
stem_words(sample)

'walk walk walk walk'

In [125]:
para = 'As the sun dipped below the horizon, painting the sky in hues of orange and pink, the children laughed and played in the park. Their joyful voices echoed through the air, blending with the soft rustling of leaves. Meanwhile, parents sat on benches, sharing stories and watching the evening unfold peacefully.'
stem_words(para)

'as the sun dip below the horizon, paint the sky in hue of orang and pink, the children laugh and play in the park. their joy voic echo through the air, blend with the soft rustl of leaves. meanwhile, parent sat on benches, share stori and watch the even unfold peacefully.'

In [126]:
# Sometime Stemmed words are not readble it just remove the ed ing like that to create the root word, 
# which can be wrong sometimes 
# And words might not be readable by human 

#### 10. Lemmetisation

In [127]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet_lemmetizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/de-ninja/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/de-ninja/nltk_data...


In [140]:
import string
sentence = 'As the sun dipped below the horizon, painting the sky in hues of orange and pink, the children laughed and played in the park.'
puntuations = string.punctuation
sentence_words = nltk.word_tokenize(sentence)

for word in sentence_words:
    if word in puntuations:
        sentence_words.remove(word)

print("{0:20} | {1:20}".format('Word',"Lemma"))
for word in sentence_words:
    print("{0:20} | {1:20}".format(word, wordnet_lemmetizer.lemmatize(word, pos='v')))

Word                 | Lemma               
As                   | As                  
the                  | the                 
sun                  | sun                 
dipped               | dip                 
below                | below               
the                  | the                 
horizon              | horizon             
painting             | paint               
the                  | the                 
sky                  | sky                 
in                   | in                  
hues                 | hue                 
of                   | of                  
orange               | orange              
and                  | and                 
pink                 | pink                
the                  | the                 
children             | children            
laughed              | laugh               
and                  | and                 
played               | play                
in                   | in       

In [141]:
# Note: Lemmetization is better than Stemming but slower as compared to stemming

#### Practice - Twitter Sentiment Dataset

In [142]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")


100%|██████████████████████████████████████████████████████████████████| 1.99M/1.99M [00:00<00:00, 2.15MB/s]

Extracting files...
Path to dataset files: /home/de-ninja/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2


In [143]:
!ls /home/de-ninja/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2

twitter_training.csv  twitter_validation.csv


In [37]:
df = pd.read_csv('/home/de-ninja/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2'+'/twitter_training.csv', header=None, names = ['sno', 'game', 'sentiment', 'review'])
df.head()

,sno,game,sentiment,review
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [38]:
df['review'] = df['review'].astype(str)

In [39]:
# 1. Removing HTML tags
import re
def remove_html_tags(text):
    pattern = re.compile(r'<.*?>')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(remove_html_tags)
df.head()

,sno,game,sentiment,review
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [40]:
# 2. Renoving the urls
def remove_urls(text):
    https_pattern = re.compile(r'https?://\S+')
    http_pattern = re.compile(r'https?://\S+')
    text = http_pattern.sub(r'', text)
    text = https_pattern.sub(r'', text)
    return text

df['review'] = df['review'].apply(remove_urls)
df.head()

,sno,game,sentiment,review
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [41]:
# 3. Removing Punctuations
import string 
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [42]:
import re
def remove_punctuations_v1(text):
    pattern = re.compile(r'[{0}]'.format(punctuations))
    return pattern.sub(r'', text)

df['review'] = df['review'].apply(remove_punctuations_v1) 
df.head()

,sno,game,sentiment,review
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [43]:
# 4. Removing Chat Abbreviations
chat_abbreviations = {
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "BRB": "Be Right Back",
    "BTW": "By The Way",
    "DM": "Direct Message",
    "FYI": "For Your Information",
    "GG": "Good Game",
    "GLHF": "Good Luck, Have Fun",
    "IDK": "I Don’t Know",
    "IMO": "In My Opinion",
    "IMHO": "In My Humble Opinion",
    "IRL": "In Real Life",
    "LMAO": "Laughing My Ass Off",
    "LOL": "Laugh Out Loud",
    "NP": "No Problem",
    "NSFW": "Not Safe For Work",
    "OMG": "Oh My God",
    "OMW": "On My Way",
    "ROFL": "Rolling On Floor Laughing",
    "SMH": "Shaking My Head",
    "TBH": "To Be Honest",
    "TTYL": "Talk To You Later",
    "TY": "Thank You",
    "YOLO": "You Only Live Once",
    "JK": "Just Kidding",
    "TMI": "Too Much Information",
    "WFH": "Work From Home",
    "LMK": "Let Me Know",
    "IKR": "I Know, Right?",
    "RN": "Right Now",
    "BFF": "Best Friends Forever",
    "FOMO": "Fear Of Missing Out",
    "GTG": "Got To Go",
    "IDC": "I Don’t Care",
    "ETA": "Estimated Time of Arrival",
    "HMU": "Hit Me Up",
    "WC": "Welcome (or Water Closet)",
    "YOLO": "You Only Live Once"
}

In [44]:
def remove_chat_abbreviations(text):
    new_text = []
    for word in text.split():
        if word.upper() in chat_abbreviations.keys():
            new_text.append(chat_abbreviations[word.upper()])
        else:
            new_text.append(word)
    return ' '.join(new_text)

df['review'] = df['review'].apply(remove_chat_abbreviations)
df.head()

,sno,game,sentiment,review
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [45]:
# 5. Handling the Incorrect Spellings
from textblob import TextBlob
def handling_incorrect_spellings(text):
    textblobObj = TextBlob(text)
    return textblobObj.correct().string

# df['review'] = df['review'].apply(handling_incorrect_spellings)
# df.head()

In [46]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Twitter_data').getOrCreate()

In [47]:
base = spark.createDataFrame(df)

In [48]:
from pyspark.sql.functions import udf

handlling_spellings = udf(handling_incorrect_spellings)
base = base.withColumn('review', handlling_spellings('review'))
base.show(5)

[Stage 6:>                                                          (0 + 1) / 1]

+----+-----------+---------+--------------------+
| sno|       game|sentiment|              review|
+----+-----------+---------+--------------------+
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|I am coming to th...|
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|in coming on bord...|
|2401|Borderlands| Positive|in getting on bor...|
+----+-----------+---------+--------------------+
only showing top 5 rows



In [49]:
# 6. Handlling Stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

en_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/de-
[nltk_data]     ninja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
%%time
# def remove_stopwords(text):
#     new_text = []
#     for word in text.split():
#         if word not in en_stopwords:
#             new_text.append(word)
#     return new_text
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in en_stopwords])

remove_stopwords_udf = udf(remove_stopwords)

base_v2 = base.withColumn('review', remove_stopwords_udf('review'))
base_v2.show(5)

[Stage 7:>                                                          (0 + 1) / 1]

+----+-----------+---------+--------------------+
| sno|       game|sentiment|              review|
+----+-----------+---------+--------------------+
|2401|Borderlands| Positive|getting borderlan...|
|2401|Borderlands| Positive|I coming borders ...|
|2401|Borderlands| Positive|getting borderlan...|
|2401|Borderlands| Positive|coming borderland...|
|2401|Borderlands| Positive|getting borderlan...|
+----+-----------+---------+--------------------+
only showing top 5 rows

CPU times: user 8.35 ms, sys: 0 ns, total: 8.35 ms
Wall time: 1.31 s


In [51]:
# 7. Handling Emojis
import re

def remove_emojis(text):
    emoji_pattern = re.compile(
        r"[\U0001F600-\U0001F64F]"  # Emoticons
        r"|[\U0001F300-\U0001F5FF]"  # Miscellaneous Symbols and Pictographs
        r"|[\U0001F680-\U0001F6FF]"  # Transport and Map Symbols
        r"|[\u2600-\u26FF]"          # Miscellaneous Symbols
        r"|[\u2700-\u27BF]"          # Dingbats
        r"|[\U0001F900-\U0001F9FF]"  # Supplemental Symbols and Pictographs
        r"|[\U0001FA70-\U0001FAFF]"  # Symbols and Pictographs Extended
        r"|[\U0001F1E6-\U0001F1FF]"  # Regional Indicator Symbols
        r"|[\U0001F200-\U0001F2FF]"  # Enclosed Ideographic Supplement
        r"|[\U0001F7E0-\U0001F7FF]"  # Geometric Shapes Extended
        r"|[\U0001F100-\U0001F1FF]"  # Enclosed Alphanumeric Supplement
        r"|[\U0001F3FB-\U0001F3FF]"  # Skin tone modifiers
    )

    return emoji_pattern.sub('', text)

removing_emojis_udf = udf(remove_emojis)
base = base.withColumn('review', removing_emojis_udf('review'))
base.show(5)

[Stage 8:>                                                          (0 + 1) / 1]

+----+-----------+---------+--------------------+
| sno|       game|sentiment|              review|
+----+-----------+---------+--------------------+
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|I am coming to th...|
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|in coming on bord...|
|2401|Borderlands| Positive|in getting on bor...|
+----+-----------+---------+--------------------+
only showing top 5 rows



In [52]:
base.show(5)

[Stage 9:>                                                          (0 + 1) / 1]

+----+-----------+---------+--------------------+
| sno|       game|sentiment|              review|
+----+-----------+---------+--------------------+
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|I am coming to th...|
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|in coming on bord...|
|2401|Borderlands| Positive|in getting on bor...|
+----+-----------+---------+--------------------+
only showing top 5 rows



In [53]:
# # 8. Tokenization
# from pyspark.sql.types import *
# import spacy
# nlp = spacy.load("en_core_web_sm")
# broadcast_nlp = spark.sparkContext.broadcast(nlp)

# # Define a UDF for tokenization
# def tokenize_text(text):
#     nlp_model = broadcast_nlp.value
#     doc = nlp_model(text)
#     return doc

# # Register the UDF
# tokenize_udf = udf(tokenize_text, StringType())

# base = base.withColumn('review_tokens', tokenize_udf('review'))
# base.show()

In [54]:
# 10. Lemmetization
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet_lemmetizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/de-ninja/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/de-ninja/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [55]:
def lemmetize_word(text):
    words = text.split()
    return ' '.join([wordnet_lemmetizer.lemmatize(word) for word in words])

lemmitizer_udf = udf(lemmetize_word)

base = base.withColumn('review', lemmitizer_udf('review'))
base.show(5)

[Stage 10:>                                                         (0 + 1) / 1]

+----+-----------+---------+--------------------+
| sno|       game|sentiment|              review|
+----+-----------+---------+--------------------+
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|I am coming to th...|
|2401|Borderlands| Positive|in getting on bor...|
|2401|Borderlands| Positive|in coming on bord...|
|2401|Borderlands| Positive|in getting on bor...|
+----+-----------+---------+--------------------+
only showing top 5 rows



In [56]:
base.show(5, False)

[Stage 11:>                                                         (0 + 1) / 1]

+----+-----------+---------+-------------------------------------------------------+
|sno |game       |sentiment|review                                                 |
+----+-----------+---------+-------------------------------------------------------+
|2401|Borderlands|Positive |in getting on borderland and i will murder you all     |
|2401|Borderlands|Positive |I am coming to the border and I will kill you all      |
|2401|Borderlands|Positive |in getting on borderland and i will kill you all       |
|2401|Borderlands|Positive |in coming on borderland and i will murder you all      |
|2401|Borderlands|Positive |in getting on borderland 2 and i will murder you me all|
+----+-----------+---------+-------------------------------------------------------+
only showing top 5 rows



In [57]:
spark.stop()